In [115]:
import pandas as pd
import numpy as np
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

from sklearn.datasets import load_breast_cancer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.datasets import load_breast_cancer

train_set = pd.read_csv('/Users/admin/Downloads/train.csv')
train_set.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [116]:
train_set.corr() #рассматриваем корреляцию между фичами

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
PassengerId,1.000000,-0.005007,-0.035144,0.036847,-0.057527,-0.001652,0.012658
Survived,-0.005007,1.000000,-0.338481,-0.077221,-0.035322,0.081629,0.257307
Pclass,-0.035144,-0.338481,1.000000,-0.369226,0.083081,0.018443,-0.549500
Age,0.036847,-0.077221,-0.369226,1.000000,-0.308247,-0.189119,0.096067
SibSp,-0.057527,-0.035322,0.083081,-0.308247,1.000000,0.414838,0.159651
Parch,-0.001652,0.081629,0.018443,-0.189119,0.414838,1.000000,0.216225
Fare,0.012658,0.257307,-0.549500,0.096067,0.159651,0.216225,1.000000


In [117]:
train_set.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [118]:
#дропаем ненужную инфу, она не влияет на предсказание модели
train_set.drop(['PassengerId','Ticket', 'Name'],axis=1,inplace=True)
train_set.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Cabin       687
Embarked      2
dtype: int64

In [120]:
train_set.drop('Cabin', axis=1,inplace=True) # из-за большого количества пустых данных и низкой корреляции с остальными фичами удаляем
train_set

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S
887,1,1,female,19.0,0,0,30.0000,S
888,0,3,female,NaN,1,2,23.4500,S
889,1,1,male,26.0,0,0,30.0000,C


In [121]:
# процент пропущенных значений для каждой колонки:
train_set.isnull().sum() / train_set.shape[0]

Survived    0.000000
Pclass      0.000000
Sex         0.000000
Age         0.198653
SibSp       0.000000
Parch       0.000000
Fare        0.000000
Embarked    0.002245
dtype: float64

In [122]:
X = train_set.drop([ 'Survived'], axis=1) 
X.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,22.0,1,0,7.2500,S
1,1,female,38.0,1,0,71.2833,C
2,3,female,26.0,0,0,7.9250,S
3,1,female,35.0,1,0,53.1000,S
4,3,male,35.0,0,0,8.0500,S


In [123]:
y = train_set.Survived

In [100]:
# Представляем пол в численном значении: male = 0, female = 1.
X = pd.get_dummies(X)
X.head()

,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,3,22.0,1,0,7.2500,0,1,0,0,1
1,1,38.0,1,0,71.2833,1,0,1,0,0
2,3,26.0,0,0,7.9250,1,0,0,0,1
3,1,35.0,1,0,53.1000,1,0,0,0,1
4,3,35.0,0,0,8.0500,0,1,0,0,1


In [101]:
# Заполним пропущенные значения возраста медианным значением возраста:
X = X.fillna({'Age': X.Age.median()})

In [102]:
# дополнительная проверка
X.isnull().sum()

Pclass        0
Age           0
SibSp         0
Parch         0
Fare          0
Sex_female    0
Sex_male      0
Embarked_C    0
Embarked_Q    0
Embarked_S    0
dtype: int64

In [103]:
clf = tree.DecisionTreeClassifier(criterion='entropy', random_state=0) # дерево решений
clf

DecisionTreeClassifier(criterion='entropy', random_state=0)

In [104]:
 # Разобьем выборку на тренировочную и тестовую:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)

In [105]:
clf.fit(X, y)

DecisionTreeClassifier(criterion='entropy', random_state=0)

In [106]:
clf.score(X_train, y_train)

0.9798657718120806

In [107]:
clf.score(X_test, y_test) # проверка на переобученность модели
clf_score = clf.score(X_test, y_test)
print("clf_score = {}%".format(clf_score * 100))

clf_score = 97.96610169491525%


In [108]:
rf = RandomForestRegressor (n_estimators=100) # метод случайного леса

In [109]:
rf.fit(X, y)

RandomForestRegressor()

In [110]:
r2_score = rf.score(X_test, y_test)
print("R2_score = {}%".format(r2_score * 100))

R2_score = 88.81867042838314%


In [111]:
# Добавим кроссвалидацию:

scores = pd.DataFrame()

for max_depth in range(1, 16):
    clf = tree.DecisionTreeClassifier(criterion='entropy', max_depth=max_depth)
    clf.fit(X_train, y_train)
    train_score = clf.score(X_train, y_train)
    test_score = clf.score(X_test, y_test)
    
    mean_cross_val_score = cross_val_score(clf, X_train, y_train, cv=5).mean()
    
    temp = pd.DataFrame({
        'max_depth': [max_depth],
        'train_score': [train_score],
        'test_score': [test_score],
        'mean_cross_val_score': [mean_cross_val_score],
    })
    scores = scores.append(temp)

scores.head()

,max_depth,train_score,test_score,mean_cross_val_score
0,1,0.788591,0.783051,0.788655
0,2,0.788591,0.783051,0.770168
0,3,0.825503,0.803390,0.805378
0,4,0.843960,0.813559,0.810420
0,5,0.848993,0.806780,0.783599


In [112]:
# Адаптивный бустинг
X, y = load_breast_cancer(return_X_y=True)

modelClf = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=2), n_estimators=100, random_state=12)

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.33, random_state=12)

modelClf.fit(X_train, y_train)
print(modelClf.score(X_valid, y_valid))

0.9627659574468085
